# GAMER Cluster Merger Problem yt analysis

This Jupyter notebook shows a quick review of the features of the `ClusterMerger` problem in GAMER using yt.

In [ ]:
import yt

The `ClusterMerger` problem adds a particle attribute, `ParType`, to distinguish between DM and star particles (so far the particle types are otherwise identical, this attribute is only here to distinguish their spatial distributions. We can use this attribute to create new particle types like this, using particle filters: 

In [ ]:
@yt.particle_filter(requires=["ParType"], filtered_type='io')
def dm(pfilter, data):
   dfilter = data[(pfilter.filtered_type, "ParType")] == 1
   return dfilter

@yt.particle_filter(requires=["ParType"], filtered_type='io')
def star(pfilter, data):
   sfilter = data[(pfilter.filtered_type, "ParType")] == 2
   return sfilter

and now when we load the dataset, we need to add these filters to it:

In [ ]:
ds = yt.load("Data_000000")
ds.add_particle_filter("dm")
ds.add_particle_filter("star")

## Special Fields

The `ClusterMerger` problem has up to four passive scalars: a metal field to track metals, and 1-3 "color" fields to track the gas from each cluster. The fields in GAMER have units of density, so if you want to look at the fractional fields, create derived fields dividing by density:

In [ ]:
def _metals(field, data):
    return data["gamer", "Metal"]/data["gamer", "Dens"].d
ds.add_field(("gas","metals"), _metals, units="", sampling_type="grid", force_override=True)
def _cluster1(field, data):
    return data["gamer", "ColorField1"]/data["gamer", "Dens"].d
ds.add_field(("gas","cluster1"), _cluster1, units="", sampling_type="grid", force_override=True)
def _cluster2(field, data):
    return data["gamer", "ColorField2"]/data["gamer", "Dens"].d
ds.add_field(("gas","cluster2"), _cluster2, units="", sampling_type="grid", force_override=True)

Now we can plot these fields up, as well as other important fields:

In [ ]:
slc = yt.SlicePlot(ds, "z", ["density", "kT", "magnetic_field_strength",
                             ("gas", "cluster2"), ("gas", "metals"), ("gas", "cluster1")],
                   width=(6.0,"Mpc"))
slc.set_log("cluster1", False)
slc.set_log("cluster2", False)

## Plotting Different Particle Types

Now we can show how the two different particle types are distributed. To plot all of the particles:

In [ ]:
p_all = yt.ParticlePlot(ds, ("io","particle_position_x"), ("io","particle_position_y"), 
                        z_fields=("io","particle_mass"), width=(6.0,"Mpc"))
p_all.show()

and plot the DM particles:

In [ ]:
p_dm = yt.ParticlePlot(ds, ("dm","particle_position_x"), ("dm","particle_position_y"), 
                       z_fields=("dm","particle_mass"), width=(6.0,"Mpc"))
p_dm.show()

and finally, the star particles:

In [ ]:
p_star = yt.ParticlePlot(ds, ("star","particle_position_x"), ("star","particle_position_y"), 
                         z_fields=("star","particle_mass"), width=(6.0,"Mpc"))
p_star.show()